In [2]:
import sqlite3 as sql

print("1 - добавление\n2 - получение")
choice = int(input("> "))
con = sql.connect('test.db')
with con:
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS `test` (`name` STRING, `surname` STRING)")

    if choice == 1:
        name = input("Name\n> ")
        surname = input("Surname\n> ")
        cur.execute(f"INSERT INTO `test` VALUES ('{name}', '{surname}')")
    elif choice == 2:
        cur.execute("SELECT * FROM `test`")
        rows = cur.fetchall()
        for row in rows:
            print(row[0], row[1])
    else:
        print("Вы ошиблись")

    con.commit()
    cur.close()

1 - добавление
2 - получение
> 1
Name
> 2
Surname
> 2


In [36]:
import sqlite3
con = sqlite3.connect("DB/Credit.db")
cur = con.cursor()
# cur.execute("CREATE TABLE credits(title, date_of_deal, body_of_credit, raid, number_of_periods, last_payment, monthly_payment)")

In [2]:
cur = con.cursor()

In [17]:
cur.execute("CREATE TABLE testing(user, title)")

In [20]:
res = cur.execute("SELECT * FROM testing")
res.fetchall()

[('u1', 1), ('u2', 2), ('u3', 3)]

In [22]:
cur.execute("""
    INSERT INTO testing VALUES
        ('u1', 1),
        ('u2', 2),
        ('u3', 3)
""")
con.commit()

In [58]:
res = cur.execute("SELECT * FROM credits")
res = res.fetchall()
print(res)

[('Ivan', 'ipoteca_1', '2022-10-09 00:00:00', 1000000, 5.0, 50, '2026-12-09 00:00:00', 22197.11), ('Ivan', 'Ipoteca_3', '2022-10-12 00:00:00', 2000000000, 5.0, 900, '2097-10-12 00:00:00', 8535642.44)]


In [15]:
cur.execute("DROP TABLE testing")

In [38]:
res = cur.execute("DELETE FROM credits WHERE title = 'ipoteca_2'")
con.commit()

In [12]:
import pandas as pd
df = pd.DataFrame(res, columns = ['title', 'year', 'score'])
df.head()

,title,year,score
0,Monty Python and the Holy Grail,1975,8.2
1,And Now for Something Completely Different,1971,7.5


In [5]:
cur.execute(f"""INSERT INTO credits VALUES
            ({name_of_credit}, {begin}, {body_of_credit}, {raid}, {number_of_periods}, {st.session_state.last_payment}, {st.session_state.monthly_payment})
""")
con.commit()

NameError: name 'name_of_credit' is not defined

In [4]:
tuples = ('Ипотека_22', '2022-09-05', 11879039, 5.7, 360, 2038, 68945.99)
def insert_to_db(cur, con, tuples):
    str_to_insert = f"INSERT INTO credits VALUES {tuples} "
    cur.execute(str_to_insert)
    con.commit()
insert_to_db(cur, con, tuples)

In [6]:
import pandas as pd
title = ['title', 'date_of_deal', 'body_of_credit', 'raid', 'number_of_periods', 'last_payment', 'monthly_payment']
pd.DataFrame(cur.execute("SELECT * FROM credits").fetchall(), columns = title).head()

,title,date_of_deal,body_of_credit,raid,number_of_periods,last_payment,monthly_payment
0,Ипотека_ЦДС,2022-09-05,11879039,5.7,360,2038,68945.99


In [54]:
import pandas as pd
import datetime as dt
pd.options.display.float_format = '{:.2f}'.format
from dateutil.relativedelta import *
last_payment = dt.datetime(2052, 9, 5)
date_now = dt.datetime(2022, 9, 5)
number_of_periods = (last_payment.year - date_now.year) * 12 + last_payment.month - date_now.month
body_credit = 11879039
raid = 5.7
monthly_raid = raid / 12 / 100
general_raid = (1 + monthly_raid) ** number_of_periods
monthly_payment = body_credit * monthly_raid * general_raid / (general_raid - 1)


def calc(date_now, body_credit, raid):
    df = []
    for month in range(0, 360):
        begin = dt.datetime(date_now.year, date_now.month, 5) + relativedelta(months=+month)
        intermediate_date = dt.datetime(date_now.year, date_now.month, 1) + relativedelta(months=+(month+1)) - relativedelta(days = 1)
        end = dt.datetime(date_now.year, date_now.month, 5) + relativedelta(months=+(month+1))

        number_of_days1 = (intermediate_date - begin).days
        number_of_days2 = (end - intermediate_date).days
        number_of_days_in_year1 = (dt.datetime(int(begin.year)+1, 1, 1) - dt.datetime(begin.year, 1, 1)).days
        number_of_days_in_year2 = (dt.datetime(int(end.year)+1, 1, 1) - dt.datetime(end.year, 1, 1)).days
        percent_part = (body_credit * raid * number_of_days1)/(100*number_of_days_in_year1) + (body_credit * raid * number_of_days2)/(100*number_of_days_in_year2)
        general_part = monthly_payment - percent_part
        if body_credit - general_part <0:
            general_part = body_credit
            body_credit = 0
        else:
            body_credit -= general_part

        df.append([end, percent_part, general_part, body_credit, number_of_days1+number_of_days2])
    df = pd.DataFrame(df, columns = ['date_payment', 'percent_part', 'general_part', 'body_of_credit', 'days'])
    return df
    
df = calc(date_now, body_credit, raid)

In [55]:
df.head(10)

,date_payment,percent_part,general_part,body_of_credit,days
0,2022-10-05,55652.48,13293.51,11865745.49,30
1,2022-11-05,57443.21,11502.78,11854242.71,31
2,2022-12-05,55536.32,13409.68,11840833.03,30
3,2023-01-05,57322.61,11623.39,11829209.65,31
4,2023-02-05,57266.34,11679.65,11817529.99,31
5,2023-03-05,51673.36,17272.63,11800257.36,28
6,2023-04-05,57126.18,11819.82,11788437.55,31
7,2023-05-05,55228.02,13717.97,11774719.58,30
8,2023-06-05,57002.55,11943.45,11762776.13,31
9,2023-07-05,55107.80,13838.19,11748937.94,30


In [57]:
#Monthly_payment -
new_payment = 5000000
date = dt.datetime(2022, 12, 15)
begin = dt.datetime(2022, 9, 5)

date_now = df[df['date_payment']<date].max()['date_payment']
number_of_periods = 360
last_payment = begin + relativedelta(months=+number_of_periods)


body_credit = df[df['date_payment']>date].max()['body_of_credit'] - new_payment
print(body_credit)
raid = 5.7
monthly_raid = raid / 12 / 100
general_raid = (1 + monthly_raid) ** number_of_periods
monthly_payment = body_credit * monthly_raid * general_raid / (general_raid - 1)

df2 = calc(date_now, body_credit, raid)
df2 = pd.concat([df[df['date_payment']<date], df2])
df2['m_pay'] = df2['percent_part'] + df2['general_part']
df2.head(10)

# df2 = calc(date_now, body_credit, raid)
# df2.head()

6829209.646515802


,date_payment,percent_part,general_part,body_of_credit,days,m_pay
0,2022-10-05,55652.48,13293.51,11865745.49,30,68945.99
1,2022-11-05,57443.21,11502.78,11854242.71,31,68945.99
2,2022-12-05,55536.32,13409.68,11840833.03,30,68945.99
0,2023-01-05,33060.86,6575.90,6822633.74,31,39636.76
1,2023-02-05,33029.02,6607.74,6816026.01,31,39636.76
2,2023-03-05,29803.77,9832.99,6806193.02,28,39636.76
3,2023-04-05,32949.43,6687.33,6799505.69,31,39636.76
4,2023-05-05,31855.22,7781.54,6791724.14,30,39636.76
5,2023-06-05,32879.39,6757.37,6784966.77,31,39636.76
6,2023-07-05,31787.10,7849.66,6777117.11,30,39636.76


In [44]:
df[df['date_payment']<date].head()

,date_payment,percent_part,general_part,body_of_credit,days
0,2022-11-05,57507.57,11511.17,11867527.83,31
1,2022-12-05,55598.56,13420.18,11854107.64,30


In [46]:
new_payment = 500000
date = dt.datetime(2022, 12, 15)

raid = 5.7
monthly_raid = raid / 12 / 100
general_raid = (1 + monthly_raid) ** number_of_periods
monthly_payment = 68945.99
body_credit = 11879039# - new_payment

In [47]:
print(monthly_raid)
print(general_raid)

0.004750000000000001
5.50662308464368


In [48]:
monthly_payment = body_credit * monthly_raid * general_raid / (general_raid - 1)

In [52]:
monthly_payment

68945.99314672532

In [49]:
mp = body_credit * monthly_raid * general_raid
print(mp)
mp2 = mp/(general_raid - 1)
print(mp2)

310713.6043087173
68945.99314672532


In [72]:
(310713.6043087173 / 68945.99314672532) +1

5.506623084643681

In [81]:
gr = (body_credit * monthly_raid)/monthly_payment
print(gr)

1.63680107429018


In [84]:
monthly_payment/(body_credit*monthly_raid)

1.2219172366461566

In [65]:
import math
math.log(general_raid, (1 + monthly_raid))

360.0

In [50]:
n1 = (1+monthly_raid)
n2 = (1-monthly_payment/(body_credit*monthly_raid))
print(n1, n2)
math.log(n1, n2)

1.00475 -0.22189563698093595


NameError: name 'math' is not defined

In [ ]:
date_now = df[df['date_payment']<date].max()['date_payment']
number_of_periods = 360
last_payment = begin + relativedelta(months=+number_of_periods)


In [ ]:
general_raid = (1 + monthly_raid) ** number_of_periods
monthly_payment = body_credit * monthly_raid * general_raid / (general_raid - 1)

In [22]:
df.to_sql('Ipoteca_CDS', con, index=False, if_exists = 'replace')#, dtype={'date_payment':BLOB, 'percent_part':float, 'general_part':float, 'body_of_credit':float, 'days':int})

360

In [23]:
res = cur.execute("SELECT * FROM Ipoteca_CDS")
res = res.fetchall()

In [25]:
pd.DataFrame(res, columns = ['date_payment', 'percent_part', 'general_part', 'body_of_credit', 'days']).head()

,date_payment,percent_part,general_part,body_of_credit,days
0,2022-10-05 00:00:00,55652.484082,13293.509065,1.186575e+07,30
1,2022-11-05 00:00:00,57443.211733,11502.781414,1.185424e+07,31
2,2022-12-05 00:00:00,55536.315160,13409.677987,1.184083e+07,30
3,2023-01-05 00:00:00,57322.608128,11623.385019,1.182921e+07,31
4,2023-02-05 00:00:00,57266.338207,11679.654940,1.181753e+07,31


In [10]:
# imports
import plotly.express as px
import pandas as pd

# dataframe
df = px.data.gapminder()
df=df.query("year==2007")
df.head(30)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
71,Australia,Oceania,2007,81.235,20434176,34435.367440,AUS,36
83,Austria,Europe,2007,79.829,8199783,36126.492700,AUT,40
95,Bahrain,Asia,2007,75.635,708573,29796.048340,BHR,48
107,Bangladesh,Asia,2007,64.062,150448339,1391.253792,BGD,50
119,Belgium,Europe,2007,79.441,10392226,33692.605080,BEL,56


In [14]:
df.iloc[0, 0]

'Afghanistan'

In [88]:
6 * (20/5)

24.0

In [92]:
r = 5.7
nop = 306
mr = r / 12 / 100
gr = (1 + mr) ** nop
mp = 68945.99
bc = 11879039

In [97]:
print(((((1+mr)**nop)-1)/((1+mr)**nop)/mp), (bc*mr)/mp)

1.1102074411078564e-05 0.81840053714509
